Projeto 2 de Teoria e Aplicação de Grafos

Semestre 25.2

Integrantes:

Davi Galvão Guerra - 241038577
Eric Wu Harris - 242001482
Felipe Lauterjung Caselli - 241032401

In [ ]:
#Importa as métricas do projeto
import re

N_PROJETOS = 50
N_ALUNOS = 200
MAX_VAGAS = 80

In [ ]:
#Define as classes necessárias para o emparelhamento
class Projeto:
    def __init__(self, codigo, n_vagas, min_notas):
        self.codigo = codigo
        self.n_vagas = n_vagas
        self.min_notas = min_notas
        self.alunos_atribuidos = []

    def __repr__(self):
        return f"Projeto('{self.codigo}', Vagas: {self.n_vagas}, Nota mínima: {self.min_notas}, Alunos: {[aluno.codigo for aluno in self.alunos_atribuidos]})"

class Aluno:
    def __init__(self, codigo, preferencias, nota):
        self.codigo = codigo
        self.preferencias = preferencias
        self.nota = nota
        self.projeto_atribuido = None

    def __repr__(self):
        return f"Aluno('{self.codigo}', Nota: {self.nota}, Prefs: {self.preferencias}, Projeto: {self.projeto_atribuido})"

In [ ]:
#Extrai as informações do "entradaProj2.25TAG.txt"
projetos = []
alunos = []

def extrair_e_popular_listas(nome_arquivo, lista_projetos, lista_alunos):
    padrao_projeto = re.compile(r"^\((P\d+), (\d+), (\d+)\)$")
    padrao_aluno = re.compile(r"^\((A\d+)\):\((.*?)\) \((\d+)\)$")

    try:
        with open(nome_arquivo, 'r', encoding='utf-8') as arquivo:
            for linha in arquivo:
                linha = linha.strip()

                if not linha or linha.startswith('//'):
                    continue

                match_projeto = padrao_projeto.match(linha)
                if match_projeto:
                    codigo = match_projeto.group(1)
                    n_vagas = int(match_projeto.group(2))
                    min_notas = int(match_projeto.group(3))

                    projeto = Projeto(codigo, n_vagas, min_notas)
                    lista_projetos.append(projeto)
                    continue

                match_aluno = padrao_aluno.match(linha)
                if match_aluno:
                    codigo = match_aluno.group(1)
                    projetos_str = match_aluno.group(2)
                    nota = int(match_aluno.group(3))

                    preferencias = [p.strip() for p in projetos_str.split(',')]

                    aluno = Aluno(codigo, preferencias, nota)
                    lista_alunos.append(aluno)
                    continue
    except Exception as e:
        print(e)

In [ ]:
extrair_e_popular_listas('/entradaProj2.25TAG.txt', projetos, alunos)

In [ ]:
#Cria o índice de preferência por projeto
preferencias_projetos = [[] for _ in range(N_PROJETOS)]
for aluno in alunos:
  for projeto_code_str in aluno.preferencias:
    projeto_idx = int(projeto_code_str[1:]) - 1
    if 0 <= projeto_idx < N_PROJETOS:
        if(projetos[projeto_idx].min_notas <= aluno.nota):
          preferencias_projetos[projeto_idx].append(aluno)
    else:
        print(f"{projeto_code_str} out of expected range for aluno {aluno.codigo}")

In [ ]:
#Organiza a ordem de preferência dos projetos
for project_idx in range(N_PROJETOS):
    project_code = f"P{project_idx + 1}"
    preferencias_projetos[project_idx].sort(key=lambda aluno: (
        aluno.preferencias.index(project_code) if project_code in aluno.preferencias else float('inf'),
        -aluno.nota,
        aluno.codigo
    ))

In [ ]:
#Algoritmo SPA
import copy

def SPA(initial_projetos, initial_alunos, preferencias_projetos):
    projetos_copy = copy.deepcopy(initial_projetos)
    alunos_copy = copy.deepcopy(initial_alunos)

    projetos_map = {p.codigo: p for p in projetos_copy}

    alunos_livres = [aluno for aluno in alunos_copy if aluno.preferencias]

    while alunos_livres:
        aluno = alunos_livres.pop(0)

        if not aluno.preferencias:
            continue

        projeto_codigo = aluno.preferencias.pop(0)

        projeto = projetos_map.get(projeto_codigo)

        if projeto:
            if aluno.nota >= projeto.min_notas:
                if len(projeto.alunos_atribuidos) < projeto.n_vagas:
                    projeto.alunos_atribuidos.append(aluno)
                    aluno.projeto_atribuido = projeto
                else:
                    project_idx = int(projeto.codigo[1:]) - 1

                    if not (0 <= project_idx < len(preferencias_projetos)):
                        if aluno.preferencias:
                            alunos_livres.append(aluno)
                        continue

                    project_prefs_list = preferencias_projetos[project_idx]

                    pior_aluno = None
                    pior_aluno_rank_index = -1

                    for assigned_aluno in projeto.alunos_atribuidos:
                        try:
                            rank = project_prefs_list.index(assigned_aluno)
                        except ValueError:
                            rank = float('inf')

                        if rank > pior_aluno_rank_index:
                            pior_aluno_rank_index = rank
                            pior_aluno = assigned_aluno

                    aluno_rank_index = float('inf')
                    try:
                        aluno_rank_index = project_prefs_list.index(aluno)
                    except ValueError:
                        if aluno.preferencias:
                            alunos_livres.append(aluno)
                        continue

                    if pior_aluno and aluno_rank_index < pior_aluno_rank_index:
                        projeto.alunos_atribuidos.remove(pior_aluno)
                        pior_aluno.projeto_atribuido = None
                        if pior_aluno.preferencias:
                            alunos_livres.append(pior_aluno)

                        projeto.alunos_atribuidos.append(aluno)
                        aluno.projeto_atribuido = projeto
                    else:
                        if aluno.preferencias:
                            alunos_livres.append(aluno)
            else:
                if aluno.preferencias:
                    alunos_livres.append(aluno)
        else:
            if aluno.preferencias:
                alunos_livres.append(aluno)

    return projetos_copy, alunos_copy

In [ ]:
final_projetos_da, final_alunos_da = SPA(projetos, alunos, preferencias_projetos)

In [ ]:
#Gera a matriz final dos emparelhamentos com as escolhas dos estudantes
import pandas as pd

matching_data = []

for aluno in final_alunos_da:
    student_code = aluno.codigo
    assigned_project_code = None
    student_rank_in_project = None
    project_rank_for_student = None

    if aluno.projeto_atribuido:
        assigned_project_code = aluno.projeto_atribuido.codigo

        project_idx = int(assigned_project_code[1:]) - 1
        if 0 <= project_idx < len(preferencias_projetos):
            try:
                temp_aluno_list_for_project_rank = [a for a in preferencias_projetos[project_idx] if a.codigo == aluno.codigo]
                if temp_aluno_list_for_project_rank:
                    student_rank_in_project = preferencias_projetos[project_idx].index(temp_aluno_list_for_project_rank[0]) + 1
                else:
                    student_rank_in_project = 'Não na Pref. do Projeto'
            except ValueError:
                student_rank_in_project = 'Não na Pref. do Projeto'
        else:
            student_rank_in_project = 'Índice de Projeto Inválido'

        original_aluno = next((a for a in alunos if a.codigo == aluno.codigo), None)
        if original_aluno and assigned_project_code in original_aluno.preferencias:
            project_rank_for_student = original_aluno.preferencias.index(assigned_project_code) + 1
        else:
            project_rank_for_student = 'Não nas Pref. do Aluno'
    else:
        assigned_project_code = 'Não Alocado'
        student_rank_in_project = None
        project_rank_for_student = None

    matching_data.append({
        'Aluno': student_code,
        'Projeto Emparelhado': assigned_project_code,
        'Rank do Aluno (pelo Projeto)': student_rank_in_project,
        'Rank do Projeto (pelo Aluno)': project_rank_for_student
    })

matching_matrix_df = pd.DataFrame(matching_data)
display(matching_matrix_df)

In [ ]:
import copy
import re
import networkx as nx
import matplotlib.pyplot as plt
import os

In [ ]:
# Redefine as classes novamente para garantir que estejam disponíveis dentro do novo escopo da função
class Projeto:
    def __init__(self, codigo, n_vagas, min_notas):
        self.codigo = codigo
        self.n_vagas = n_vagas
        self.min_notas = min_notas
        self.alunos_atribuidos = []

    def __repr__(self):
        return f"Projeto('{self.codigo}', Vagas: {self.n_vagas}, Nota mínima: {self.min_notas}, Alunos: {[aluno.codigo for aluno in self.alunos_atribuidos]})"

class Aluno:
    def __init__(self, codigo, preferencias, nota):
        self.codigo = codigo
        self.preferencias = preferencias 
        self.original_preferencias = list(preferencias)
        self.nota = nota
        self.projeto_atribuido = None

    def __repr__(self):
        return f"Aluno('{self.codigo}', Nota: {self.nota}, Prefs: {self.preferencias}, Projeto: {self.projeto_atribuido})"

In [ ]:
# Função SPA com visualização do histórico
def SPA_visualize(initial_projetos_map, initial_alunos, preferencias_projetos_original):
    projetos_copy = copy.deepcopy(list(initial_projetos_map.values())) 
    alunos_copy = copy.deepcopy(initial_alunos)

    for i, aluno_orig in enumerate(initial_alunos):
        alunos_copy[i].preferencias = list(aluno_orig.preferencias)
        alunos_copy[i].original_preferencias = list(aluno_orig.preferencias)

    preferencias_projetos_viz = [[a for a in sublist] for sublist in preferencias_projetos_original]

    projetos_map = {p.codigo: p for p in projetos_copy}
    alunos_map = {a.codigo: a for a in alunos_copy}

    alunos_livres_queue = [aluno for aluno in alunos_copy if aluno.preferencias]

    history = []
    iteration = 0
    rejected_edges_set = set()

    while alunos_livres_queue:
        iteration += 1
        current_matchings = []
        for p_code, p_obj in projetos_map.items():
            for a in p_obj.alunos_atribuidos:
                current_matchings.append((a.codigo, p_obj.codigo))

        history.append({
            'iteration': iteration,
            'proposing': None,
            'matchings': list(current_matchings),
            'free_students': [a.codigo for a in alunos_livres_queue],
            'rejection': None,
            'all_rejected_edges': list(rejected_edges_set)
        })

        aluno = alunos_livres_queue.pop(0)

        if not aluno.preferencias:
            continue

        projeto_codigo = aluno.preferencias.pop(0)
        projeto = projetos_map.get(projeto_codigo)
        proposing_edge = (aluno.codigo, projeto_codigo)

        history[-1]['proposing'] = proposing_edge

        if projeto:
            if aluno.nota >= projeto.min_notas: 
                if len(projeto.alunos_atribuidos) < projeto.n_vagas:
                    projeto.alunos_atribuidos.append(aluno)
                    aluno.projeto_atribuido = projeto
                else:
                    project_idx = int(projeto.codigo[1:]) - 1 

                    if not (0 <= project_idx < len(preferencias_projetos_viz)):
                        if aluno.preferencias:
                            alunos_livres_queue.append(aluno)
                        rejected_edges_set.add(proposing_edge)
                        continue

                    project_prefs_list = preferencias_projetos_viz[project_idx]

                    pior_aluno = None
                    pior_aluno_rank = -1

                    assigned_alunos_codes = {a.codigo for a in projeto.alunos_atribuidos}

                    for pref_aluno_obj in project_prefs_list:
                        if pref_aluno_obj.codigo in assigned_alunos_codes:
                            try:
                                current_rank = project_prefs_list.index(pref_aluno_obj)
                            except ValueError:
                                current_rank = float('inf')
                            if pior_aluno is None or current_rank > pior_aluno_rank:
                                pior_aluno_rank = current_rank
                                pior_aluno = pref_aluno_obj

                    aluno_rank = float('inf')
                    for pref_aluno_obj in project_prefs_list:
                        if pref_aluno_obj.codigo == aluno.codigo:
                            aluno_rank = project_prefs_list.index(pref_aluno_obj)
                            break

                    if pior_aluno and aluno_rank < pior_aluno_rank:
                        projeto.alunos_atribuidos.remove(alunos_map[pior_aluno.codigo])
                        alunos_map[pior_aluno.codigo].projeto_atribuido = None

                        if alunos_map[pior_aluno.codigo].preferencias:
                            alunos_livres_queue.append(alunos_map[pior_aluno.codigo])

                        rejection_edge = (alunos_map[pior_aluno.codigo].codigo, projeto_codigo)
                        history[-1]['rejection'] = rejection_edge
                        rejected_edges_set.add(rejection_edge)

                        projeto.alunos_atribuidos.append(aluno)
                        aluno.projeto_atribuido = projeto
                    else:
                        rejection_edge = proposing_edge
                        history[-1]['rejection'] = rejection_edge
                        rejected_edges_set.add(rejection_edge)
                        if aluno.preferencias:
                            alunos_livres_queue.append(aluno)
            else:
                rejection_edge = proposing_edge
                history[-1]['rejection'] = rejection_edge
                rejected_edges_set.add(rejection_edge) 
                if aluno.preferencias:
                    alunos_livres_queue.append(aluno)
        else:
            rejection_edge = proposing_edge
            history[-1]['rejection'] = rejection_edge
            rejected_edges_set.add(rejection_edge)
            if aluno.preferencias:
                alunos_livres_queue.append(aluno)

    iteration += 1 #
    final_matchings = []
    for p_code, p_obj in projetos_map.items():
        for a in p_obj.alunos_atribuidos:
            final_matchings.append((a.codigo, p_obj.codigo))

    history.append({
        'iteration': iteration,
        'proposing': None,
        'matchings': final_matchings,
        'free_students': [a.codigo for a in alunos_copy if a.projeto_atribuido is None],
        'rejection': None,
        'all_rejected_edges': list(rejected_edges_set)
    })

    return projetos_copy, alunos_copy, history

In [ ]:
# Função para desenhar o grafo bipartido em cada iteração
def draw_bipartite_graph(iteration, proposing_edge, current_matchings, rejection_edge, all_rejected_edges_for_frame, student_nodes, project_nodes, pos, output_directory):
    os.makedirs(output_directory, exist_ok=True)

    G = nx.Graph()
    G.add_nodes_from(student_nodes, bipartite=0)
    G.add_nodes_from(project_nodes, bipartite=1)

    edge_info = {}

    for student, project in current_matchings:
        edge_info[(student, project)] = 'blue'

    if proposing_edge:
        edge_info[proposing_edge] = 'green'

    if rejection_edge:
        edge_info[rejection_edge] = 'red'

    edges = []
    edge_colors = []
    for edge, color in edge_info.items():
        edges.append(edge)
        edge_colors.append(color)

    fig, ax = plt.subplots(figsize=(15, 8))

    nx.draw_networkx_nodes(G, pos, nodelist=student_nodes, node_color='lightcoral', node_shape='o', label='Alunos')
    nx.draw_networkx_nodes(G, pos, nodelist=project_nodes, node_color='lightskyblue', node_shape='s', label='Projetos')

    valid_edges = []
    valid_edge_colors = []
    for i, (u, v) in enumerate(edges):
        if u in pos and v in pos:
            valid_edges.append((u, v))
            valid_edge_colors.append(edge_colors[i])

    nx.draw_networkx_edges(G, pos, edgelist=valid_edges, edge_color=valid_edge_colors, width=2.5)

    nx.draw_networkx_labels(G, pos, font_size=8)

    action_text = ""
    action_color = 'black'

    if proposing_edge:
        aluno, projeto = proposing_edge
        action_text = f"PROPOSTA: {aluno} tenta emparelhar com {projeto}."
        action_color = 'green'

    if rejection_edge:
        aluno_rejeitado, projeto_rejeitou = rejection_edge
        if proposing_edge == rejection_edge:
            action_text = f"REJEIÇÃO: {aluno_rejeitado} é rejeitado por {projeto_rejeitou}."
            action_color = 'red'
        else:
            aluno_proponente = proposing_edge[0] if proposing_edge else '?'
            action_text = f"REJEIÇÃO: {aluno_rejeitado} é substituído e rejeitado por {projeto_rejeitou} (em favor de {aluno_proponente})."
            action_color = 'red'

    ax.set_title(f'Algoritmo Gale-Shapley - Iteração {iteration}', fontsize=16)

    if action_text:
        plt.figtext(0.5, 0.06, action_text, ha='center', fontsize=12, color=action_color)

    ax.set_title(f'Gale-Shapley Algorithm - Iteration {iteration}', fontsize=16)

    blue_line = plt.Line2D([0], [0], color='blue', lw=2.5, label='Emparelhamento Temporário')
    green_line = plt.Line2D([0], [0], color='green', lw=2.5, label='Proposta Ativa')
    red_line = plt.Line2D([0], [0], color='red', lw=2.5, label='Rejeição')

    ax.legend(handles=[green_line, blue_line, red_line], loc='upper left', bbox_to_anchor=(1,1))

    ax.set_xticks([])
    ax.set_yticks([])

    filename = os.path.join(output_directory, f'iteration_{iteration:04d}.png')
    plt.savefig(filename, bbox_inches='tight', dpi=100)
    plt.close(fig)

    print(f"Iteration {iteration:04d} visualization saved to {filename}")


In [ ]:
# Função para extrair dados do arquivo e popular as listas de projetos e alunos
def extrair_e_popular_listas(nome_arquivo, projetos_dict, lista_alunos):
    padrao_projeto = re.compile(r"^\((P\d+), (\d+), (\d+)\)$")
    padrao_aluno = re.compile(r"^\((A\d+)\):\((.*?)\) \((\d+)\)$")

    try:
        with open(nome_arquivo, 'r', encoding='utf-8') as arquivo:
            for linha in arquivo:
                linha = linha.strip()

                if not linha or linha.startswith('//'):
                    continue

                match_projeto = padrao_projeto.match(linha)
                if match_projeto:
                    codigo = match_projeto.group(1)
                    n_vagas = int(match_projeto.group(2))
                    min_notas = int(match_projeto.group(3))

                    projetos_dict[codigo] = Projeto(codigo, n_vagas, min_notas)
                    continue

                match_aluno = padrao_aluno.match(linha)
                if match_aluno:
                    codigo = match_aluno.group(1)
                    projetos_str = match_aluno.group(2)
                    nota = int(match_aluno.group(3))

                    preferencias = [p.strip() for p in projetos_str.split(',')]

                    aluno = Aluno(codigo, preferencias, nota)
                    lista_alunos.append(aluno)
                    continue
    except Exception as e:
        print(e)

In [ ]:
projetos_map = {}
alunos = []

extrair_e_popular_listas('/entradaProj2.25TAG.txt', projetos_map, alunos)

max_project_number = 0
for p_code in projetos_map.keys():
    max_project_number = max(max_project_number, int(p_code[1:]))

for aluno in alunos:
    for pref_code in aluno.preferencias:
        if pref_code.startswith('P') and len(pref_code) > 1 and pref_code[1:].isdigit():
            max_project_number = max(max_project_number, int(pref_code[1:]))

N_PROJETOS = max_project_number

N_ALUNOS = len(alunos)

preferencias_projetos = [[] for _ in range(N_PROJETOS)]
for aluno in alunos:
  for project_code_str in aluno.preferencias:
    project_idx = int(project_code_str[1:]) - 1
    if 0 <= project_idx < N_PROJETOS and project_code_str in projetos_map:
        project_obj = projetos_map[project_code_str]
        if(project_obj.min_notas <= aluno.nota):
          preferencias_projetos[project_idx].append(aluno)

for project_idx in range(N_PROJETOS):
    project_code = f"P{project_idx + 1}"
    if project_code in projetos_map:
        preferencias_projetos[project_idx].sort(key=lambda aluno_obj: (
            aluno_obj.preferencias.index(project_code) if project_code in aluno_obj.preferencias else float('inf'),
            -aluno_obj.nota,
            aluno_obj.codigo
        ))

student_nodes = [f'A{i+1}' for i in range(N_ALUNOS)]
project_nodes = [f'P{i+1}' for i in range(N_PROJETOS)]

pos = {}

student_spacing = 5
total_student_width = (len(student_nodes) - 1) * student_spacing
student_offset_x = -total_student_width / 2

for i, student in enumerate(student_nodes):
    pos[student] = (student_offset_x + i * student_spacing, 1)

project_spacing = 20
total_project_width = (len(project_nodes) - 1) * project_spacing
project_offset_x = -total_project_width / 2

for i, project in enumerate(project_nodes):
    pos[project] = (project_offset_x + i * project_spacing, 0)


_, _, history = SPA_visualize(projetos_map, alunos, preferencias_projetos)

output_directory = 'gale_shapley_frames_new_layout'


num_frames_to_generate = None

if num_frames_to_generate is None:
    frames_to_process = history
else:
    frames_to_process = history[:num_frames_to_generate]

print(f"Generating {len(frames_to_process)} visualization frames with new layout...")

for i, entry in enumerate(frames_to_process):
    current_iteration = entry['iteration']
    proposing_edge = entry['proposing']
    current_matchings = entry['matchings']
    rejection_edge = entry['rejection']
    all_rejected_edges = entry['all_rejected_edges'] 

    draw_bipartite_graph(current_iteration, proposing_edge, current_matchings, rejection_edge, all_rejected_edges, student_nodes, project_nodes, pos, output_directory)

Generating 503 visualization frames with new layout...
Iteration 0001 visualization saved to gale_shapley_frames_new_layout/iteration_0001.png
Iteration 0002 visualization saved to gale_shapley_frames_new_layout/iteration_0002.png
Iteration 0003 visualization saved to gale_shapley_frames_new_layout/iteration_0003.png
Iteration 0004 visualization saved to gale_shapley_frames_new_layout/iteration_0004.png
Iteration 0005 visualization saved to gale_shapley_frames_new_layout/iteration_0005.png
Iteration 0006 visualization saved to gale_shapley_frames_new_layout/iteration_0006.png
Iteration 0007 visualization saved to gale_shapley_frames_new_layout/iteration_0007.png
Iteration 0008 visualization saved to gale_shapley_frames_new_layout/iteration_0008.png
Iteration 0009 visualization saved to gale_shapley_frames_new_layout/iteration_0009.png
Iteration 0010 visualization saved to gale_shapley_frames_new_layout/iteration_0010.png
Iteration 0011 visualization saved to gale_shapley_frames_new_lay

In [56]:
#Cria o GIF
import imageio.v2 as imageio
import glob

output_directory = 'gale_shapley_frames_new_layout'
image_files = sorted(glob.glob(os.path.join(output_directory, '*.png')))

gif_filename = 'gale_shapley_animation_new_layout.gif'

frame_duration = 0.5

print(f"Creating GIF animation from {len(image_files)} frames with new layout...")

with imageio.get_writer(gif_filename, mode='I', duration=frame_duration) as writer:
    for filename in image_files:
        image = imageio.imread(filename)
        writer.append_data(image)

print(f"GIF animation saved as '{gif_filename}'")

Creating GIF animation from 503 frames with new layout...
GIF animation saved as 'gale_shapley_animation_new_layout.gif'
